In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [22]:
import requests
import s3fs
import pandas as pd

from datetime import timedelta, datetime
import pendulum

import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker


from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData
#from extractMOCData import read_moc 

In [4]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [8]:
fs = s3fs.S3FileSystem(anon=False)
raw_moc_lst = fs.ls("tsx-moc")
raw_moc_lst

['tsx-moc/moc_20200324.csv',
 'tsx-moc/moc_20200325.csv',
 'tsx-moc/moc_20200326.csv',
 'tsx-moc/moc_20200327.csv',
 'tsx-moc/moc_20200330.csv',
 'tsx-moc/moc_tsx_20200331.csv',
 'tsx-moc/moc_tsx_20200401.csv',
 'tsx-moc/moc_tsx_20200402.csv',
 'tsx-moc/moc_tsx_20200403.csv']

In [6]:
fl_to_get =  raw_moc_lst[-1]
flpth = f"s3://{fl_to_get}"

moc_df = pd.read_csv(flpth)

In [7]:
moc_df

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date
0,AAV,BUY,600,1.585,20200403
1,ABX,BUY,70924,28.275,20200403
2,ACB,BUY,250260,1.125,20200403
3,ACO.X,BUY,16141,35.120,20200403
4,AD,BUY,623,7.450,20200403
...,...,...,...,...,...
393,WFS,SELL,100,1.025,20200403
394,WFT,SELL,437,24.650,20200403
395,WPK,SELL,1631,41.960,20200403
396,X,SELL,18067,104.225,20200403


In [21]:
# Create price data _list
moc_key_df =  moc_df.assign(yahoo_tsx_symbol=moc_df["Symbol"].apply(lambda x: f"{x}.TO"))
moc_key_df

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date,yahoo_tsx_symbol
0,AAV,BUY,600,1.585,20200403,AAV.TO
1,ABX,BUY,70924,28.275,20200403,ABX.TO
2,ACB,BUY,250260,1.125,20200403,ACB.TO
3,ACO.X,BUY,16141,35.120,20200403,ACO.X.TO
4,AD,BUY,623,7.450,20200403,AD.TO
...,...,...,...,...,...,...
393,WFS,SELL,100,1.025,20200403,WFS.TO
394,WFT,SELL,437,24.650,20200403,WFT.TO
395,WPK,SELL,1631,41.960,20200403,WPK.TO
396,X,SELL,18067,104.225,20200403,X.TO


In [ ]:
pd.concat([df, df.textcol.apply(lambda s: pd.Series({'feature1':s+1, 'feature2':s-1}))], axis=1)

In [27]:
def get_eod_data(symbol):
    sym = yf.Ticker(symbol)
    df["currency"] = sym.info["currency"]
    
moc_key_df.apply(get_eod_data, axis=1) 

AttributeError: 'Series' object has no attribute 'upper'

In [ ]:
data = yf.download(moc_key_df["yahoo_tsx_symbol"], start="2017-01-01", end="2017-04-30")


In [13]:
bb = yf.Ticker("BB.TO")

In [25]:
bb.

'CAD'

In [15]:
bb.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1997-12-15,0.0019,0.0019,0.0018,0.0019,403200,0.0,0.0
1997-12-16,0.0019,0.0019,0.0018,0.0019,898200,0.0,0.0
1997-12-17,0.0019,0.0020,0.0018,0.0018,242400,0.0,0.0
1997-12-18,0.0018,0.0018,0.0017,0.0018,187800,0.0,0.0
1997-12-19,0.0018,0.0018,0.0017,0.0017,1329000,0.0,0.0
...,...,...,...,...,...,...,...
2020-03-30,5.3700,5.5500,5.2100,5.5400,1911200,0.0,0.0
2020-03-31,5.7900,6.0900,5.6800,5.7700,3355200,0.0,0.0
2020-04-01,4.8400,5.2300,4.6900,4.7400,5601700,0.0,0.0


In [ ]:
tsxMocData.write_tsx_moc(moc_df)

In [ ]:
# Read MOC raw data
raw_moc_flpth = cfg.TSX_MOC_DATA["MOC_DATA_FLPTH"] 

In [ ]:
raw_moc_flpth

In [ ]:
pre_client = Client()
project_nm = "MOC"

In [ ]:
try:
    proj_id = pre_client.create_project(project_name=project_nm)
except Exception as err:
    print(err.with_traceback)

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
def retrieve_url(url='https://api.tmxmoney.com/mocimbalance/en/TSX/moc.html'):
    
    html = requests.get(url).content
    df_list = pd.read_html(html, header=[0], displayed_only=False)
    moc_df = df_list[-1]
    
    moc_df["moc_date"] = dt.datetime.today().strftime('%Y%m%d')
    return moc_df

In [ ]:
moc_df = moc_to_df()

In [ ]:


today = pd.Timestamp('today')
moc_df.to_csv(f"moc_{today:%Y%m%d}.csv", index=False)

In [ ]:
moc_df.columns


In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")